In [5]:
from torch.optim import lr_scheduler
import sys
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F
from torch import Tensor
from typing import List, Callable

# 加载规范化数据集
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_dataset = ImageFolder(r'.\projectdataset\masked\train', transform=train_transform)
val_dataset = ImageFolder(r'.\projectdataset\masked\test', transform=val_transform)
train_num =  len(train_dataset)
val_num = len(val_dataset)

batch_size =4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print("Loading completed")





Loading completed


In [6]:
# model
def channel_shuffle(x: Tensor, groups: int) -> Tensor:

    batch_size, num_channels, height, width = x.size()
    channels_per_group = num_channels // groups

    # reshape
    # [batch_size, num_channels, height, width] -> [batch_size, groups, channels_per_group, height, width]
    x = x.view(batch_size, groups, channels_per_group, height, width)

    x = torch.transpose(x, 1, 2).contiguous()

    # flatten
    x = x.view(batch_size, -1, height, width)

    return x


class InvertedResidual(nn.Module):
    def __init__(self, input_c: int, output_c: int, stride: int):
        super(InvertedResidual, self).__init__()

        if stride not in [1, 2]:
            raise ValueError("illegal stride value.")
        self.stride = stride

        assert output_c % 2 == 0
        branch_features = output_c // 2
        # 当stride为1时，input_channel应该是branch_features的两倍
        # python中 '<<' 是位运算，可理解为计算×2的快速方法
        assert (self.stride != 1) or (input_c == branch_features << 1)

        if self.stride == 2:
            self.branch1 = nn.Sequential(
                self.depthwise_conv(input_c, input_c, kernel_s=3, stride=self.stride, padding=1),
                nn.BatchNorm2d(input_c),
                nn.Conv2d(input_c, branch_features, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(branch_features),
                nn.ReLU(inplace=True)
            )
        else:
            self.branch1 = nn.Sequential()

        self.branch2 = nn.Sequential(
            nn.Conv2d(input_c if self.stride > 1 else branch_features, branch_features, kernel_size=1,
                      stride=1, padding=0, bias=False),
            nn.BatchNorm2d(branch_features),
            nn.ReLU(inplace=True),
            self.depthwise_conv(branch_features, branch_features, kernel_s=3, stride=self.stride, padding=1),
            nn.BatchNorm2d(branch_features),
            nn.Conv2d(branch_features, branch_features, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(branch_features),
            nn.ReLU(inplace=True)
        )

    @staticmethod
    def depthwise_conv(input_c: int,
                       output_c: int,
                       kernel_s: int,
                       stride: int = 1,
                       padding: int = 0,
                       bias: bool = False) -> nn.Conv2d:
        return nn.Conv2d(in_channels=input_c, out_channels=output_c, kernel_size=kernel_s,
                         stride=stride, padding=padding, bias=bias, groups=input_c)

    def forward(self, x: Tensor) -> Tensor:
        if self.stride == 1:
            x1, x2 = x.chunk(2, dim=1)
            out = torch.cat((x1, self.branch2(x2)), dim=1)
        else:
            out = torch.cat((self.branch1(x), self.branch2(x)), dim=1)

        out = channel_shuffle(out, 2)

        return out


class ShuffleNetV2(nn.Module):
    def __init__(self,
                 stages_repeats: List[int],
                 stages_out_channels: List[int],
                 num_classes: int = 1000,
                 inverted_residual: Callable[..., nn.Module] = InvertedResidual):
        super(ShuffleNetV2, self).__init__()

        if len(stages_repeats) != 3:
            raise ValueError("expected stages_repeats as list of 3 positive ints")
        if len(stages_out_channels) != 5:
            raise ValueError("expected stages_out_channels as list of 5 positive ints")
        self._stage_out_channels = stages_out_channels

        # input RGB image
        input_channels = 3
        output_channels = self._stage_out_channels[0]

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True)
        )
        input_channels = output_channels

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Static annotations for mypy
        self.stage2: nn.Sequential
        self.stage3: nn.Sequential
        self.stage4: nn.Sequential

        stage_names = ["stage{}".format(i) for i in [2, 3, 4]]
        for name, repeats, output_channels in zip(stage_names, stages_repeats,
                                                  self._stage_out_channels[1:]):
            seq = [inverted_residual(input_channels, output_channels, 2)]
            for i in range(repeats - 1):
                seq.append(inverted_residual(output_channels, output_channels, 1))
            setattr(self, name, nn.Sequential(*seq))
            input_channels = output_channels

        output_channels = self._stage_out_channels[-1]
        self.conv5 = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True)
        )

        self.fc = nn.Linear(output_channels, num_classes)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.conv5(x)
        x = x.mean([2, 3])  # global pool
        x = self.fc(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def shufflenet_v2_x0_5(num_classes=1000):
    """
    Constructs a ShuffleNetV2 with 0.5x output channels, as described in
    `"ShuffleNet V2: Practical Guidelines for Efficient CNN Architecture Design"
    <https://arxiv.org/abs/1807.11164>`.
    weight: https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth

    :param num_classes:
    :return:
    """
    model = ShuffleNetV2(stages_repeats=[4, 8, 4],
                         stages_out_channels=[24, 48, 96, 192, 1024],
                         num_classes=num_classes)

    return model


def shufflenet_v2_x1_0(num_classes=1000):
    """
    Constructs a ShuffleNetV2 with 1.0x output channels, as described in
    `"ShuffleNet V2: Practical Guidelines for Efficient CNN Architecture Design"
    <https://arxiv.org/abs/1807.11164>`.
    weight: https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth

    :param num_classes:
    :return:
    """
    model = ShuffleNetV2(stages_repeats=[4, 8, 4],
                         stages_out_channels=[24, 116, 232, 464, 1024],
                         num_classes=num_classes)

    return model


def shufflenet_v2_x1_5(num_classes=1000):
    """
    Constructs a ShuffleNetV2 with 1.0x output channels, as described in
    `"ShuffleNet V2: Practical Guidelines for Efficient CNN Architecture Design"
    <https://arxiv.org/abs/1807.11164>`.
    weight: https://download.pytorch.org/models/shufflenetv2_x1_5-3c479a10.pth

    :param num_classes:
    :return:
    """
    model = ShuffleNetV2(stages_repeats=[4, 8, 4],
                         stages_out_channels=[24, 176, 352, 704, 1024],
                         num_classes=num_classes)

    return model


def shufflenet_v2_x2_0(num_classes=1000):
    """
    Constructs a ShuffleNetV2 with 1.0x output channels, as described in
    `"ShuffleNet V2: Practical Guidelines for Efficient CNN Architecture Design"
    <https://arxiv.org/abs/1807.11164>`.
    weight: https://download.pytorch.org/models/shufflenetv2_x2_0-8be3c8ee.pth

    :param num_classes:
    :return:
    """
    model = ShuffleNetV2(stages_repeats=[4, 8, 4],
                         stages_out_channels=[24, 244, 488, 976, 2048],
                         num_classes=num_classes)

    return model

In [9]:
# train
def train(dataloader, model, loss_fn, optimizer):
    loss, current, n = 0.0, 0.0, 0
    for batch, (x, y) in enumerate(dataloader):
        image, y = x.to(device), y.to(device)
        output = model(image)
        cur_loss = loss_fn(output, y)
        _, pred = torch.max(output, axis=1)
        cur_acc = torch.sum(y==pred) / output.shape[0]

        # 反向传播
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        loss += cur_loss.item()
        current += cur_acc.item()
        n = n+1

    train_loss = loss / n
    train_acc = current / n
    print('train_loss' + str(train_loss))
    print('train_acc' + str(train_acc))
    return train_loss, train_acc

# 定义一个验证函数
def val(dataloader, model, loss_fn):
    # 将模型转化为验证模型
    model.eval()
    loss, current, n = 0.0, 0.0, 0
    with torch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            image, y = x.to(device), y.to(device)
            output = model(image)
            cur_loss = loss_fn(output, y)
            _, pred = torch.max(output, axis=1)
            cur_acc = torch.sum(y == pred) / output.shape[0]
            loss += cur_loss.item()
            current += cur_acc.item()
            n = n + 1

    val_loss = loss / n
    val_acc = current / n
    print('val_loss' + str(val_loss))
    print('val_acc' + str(val_acc))
    return val_loss, val_acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

# 实例化model
model =  shufflenet_v2_x1_0(num_classes=5).to(device)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
#lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
#checkpoint = torch.load(r"C:\Users\2021\Desktop\lib8\save_model\vgg11-bbd30ac9.pth")
#model.load_state_dict(checkpoint)
loss_train = []
acc_train = []
loss_val = []
acc_val = []
epochs = 1
for t in range(epochs):
    # lr_scheduler.step()
    print(f"epoch{t+1}\n-----------")
    train_loss, train_acc = train(train_dataloader, model, loss_function, optimizer)
    val_loss, val_acc = val(val_dataloader, model, loss_function)

    loss_train.append(train_loss)
    acc_train.append(train_acc)
    loss_val.append(val_loss)
    acc_val.append(val_acc)

    # 保存最好的模型权重
    # if val_acc >min_acc:
    print(f"save model, 第{t+1}轮")
    torch.save(model.state_dict(), r'./save_model/epoch_'+str(t)+'_best_model.pth')


print('Finished Training')

using cuda device.
epoch1
-----------
train_loss0.7684843143224717
train_acc0.5385
val_loss0.6995924656391144
val_acc0.568
save model, 第1轮
Finished Training


In [10]:
#test
print('start test')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
dataset = ImageFolder(r'.\projectdataset\masked\test', transform=train_transform)
datasetloader = DataLoader(dataset, batch_size=32, shuffle=True)

# create model
# model = vgg(model_name="vgg11", num_classes=1000).to(device)
weights_path = r".\save_model\epoch_0_best_model.pth"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

top1 =0
top5= 0
data_num = len(dataset)
acc_num =0;
wrong_num = 0;
for i in range(data_num):
    x, y = dataset[i][0], dataset[i][1]
    x = torch.unsqueeze(x, dim=0)

    with torch.no_grad():
        # predict class
        output = torch.squeeze(model(x.to(device))).cpu()
        predict = torch.softmax(output, dim=0)

        predict_cla = torch.argmax(predict).numpy()
        print("predict:",predict_cla,"actual:",y)
        if predict_cla == y:
            acc_num +=1
wrong_num = data_num-acc_num
print("正确数量：",acc_num)
print("错误数量：",wrong_num)


def evaluteTop1(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
    return correct / total


def evaluteTop5(model, loader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            maxk = max((1, 5))
            y_resize = y.view(-1, 1)
            _, pred = logits.topk(maxk, 1, True, True)
            correct += torch.eq(pred, y_resize).sum().float().item()
    return correct / total
print("top1 accuracy: ",evaluteTop1(model,datasetloader))
print("top5 accuracy: ",evaluteTop5(model,datasetloader))

start test
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
pr

predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 ac